# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,berbera,10.4396,45.0143,29.61,71,1,2.31,SO,1697479411
1,1,waitangi,-43.9535,-176.5597,12.68,90,75,3.58,NZ,1697479412
2,2,ushuaia,-54.8000,-68.3000,8.81,54,75,4.63,AR,1697479323
3,3,isafjordur,66.0755,-23.1240,3.02,75,20,5.66,IS,1697479413
4,4,islamkot,24.6997,70.1770,26.94,67,0,4.80,PK,1697479378


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles ="OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
uppertemp_df = city_data_df.loc[city_data_df["Max Temp"] < 27]
lowertemp_df = uppertemp_df.loc[uppertemp_df["Max Temp"] > 21]
wind_df = lowertemp_df.loc[lowertemp_df["Wind Speed"] < 4.5]
ideal_weather_df = wind_df.loc[wind_df["Cloudiness"] ==0]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,malaga,36.7202,-4.4203,25.39,79,0,3.60,ES,1697479379
15,15,al bardiyah,31.7561,25.0865,22.87,73,0,4.47,LY,1697479416
67,67,ojinaga,29.5667,-104.4167,25.87,31,0,2.57,MX,1697479426
106,106,moires,35.0532,24.8744,22.42,87,0,0.89,GR,1697479435
323,323,canyon lake,29.8752,-98.2625,22.84,39,0,4.02,US,1697479488
400,400,zeribet el oued,34.6828,6.5111,25.94,21,0,1.72,DZ,1697479532
401,401,benghazi,32.1167,20.0667,24.35,68,0,3.93,LY,1697479532
446,446,al muwayh,22.4333,41.7583,26.73,15,0,4.14,SA,1697479544
460,460,siwa oasis,29.2041,25.5195,25.44,34,0,2.58,EG,1697479551
476,476,letlhakeng,-24.0944,25.0298,23.52,17,0,4.06,BW,1697479554


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
    "City": ideal_weather_df["City"],
    "Country": ideal_weather_df["Country"],
    "Lat": ideal_weather_df["Lat"],
    "Lng": ideal_weather_df["Lng"],
    "Humidity": ideal_weather_df["Humidity"]
})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,malaga,ES,36.7202,-4.4203,79,
15,al bardiyah,LY,31.7561,25.0865,73,
67,ojinaga,MX,29.5667,-104.4167,31,
106,moires,GR,35.0532,24.8744,87,
323,canyon lake,US,29.8752,-98.2625,39,
400,zeribet el oued,DZ,34.6828,6.5111,21,
401,benghazi,LY,32.1167,20.0667,68,
446,al muwayh,SA,22.4333,41.7583,15,
460,siwa oasis,EG,29.2041,25.5195,34,
476,letlhakeng,BW,-24.0944,25.0298,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
malaga - nearest hotel: Petit Palace Plaza Málaga
al bardiyah - nearest hotel: فندق البردي
ojinaga - nearest hotel: Hotel
moires - nearest hotel: Hotel Olympic
canyon lake - nearest hotel: Horseshoe Riverside Lodge
zeribet el oued - nearest hotel: No hotel found
benghazi - nearest hotel: ترجاب
al muwayh - nearest hotel: فندق ومطعم الموية
siwa oasis - nearest hotel: فندق الكيلانى
letlhakeng - nearest hotel: No hotel found
mizdah - nearest hotel: No hotel found
el granada - nearest hotel: Beach House
fort irwin - nearest hotel: Landmark Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
10,malaga,ES,36.7202,-4.4203,79,Petit Palace Plaza Málaga
15,al bardiyah,LY,31.7561,25.0865,73,فندق البردي
67,ojinaga,MX,29.5667,-104.4167,31,Hotel
106,moires,GR,35.0532,24.8744,87,Hotel Olympic
323,canyon lake,US,29.8752,-98.2625,39,Horseshoe Riverside Lodge
400,zeribet el oued,DZ,34.6828,6.5111,21,No hotel found
401,benghazi,LY,32.1167,20.0667,68,ترجاب
446,al muwayh,SA,22.4333,41.7583,15,فندق ومطعم الموية
460,siwa oasis,EG,29.2041,25.5195,34,فندق الكيلانى
476,letlhakeng,BW,-24.0944,25.0298,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles ="OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)